In [1]:
cd ..

/home/yann/IVISpaces


In [7]:
import math

import torch
from torch import nn

import matplotlib.pyplot as plt

from torch.utils.data import Dataset

from Metrics import evaluate_metrics
from Experiments import get_setup


In [8]:

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')



# Setup #

In [9]:
#toy datasets : ['foong','foong_sparse', 'foong_mixed','foong2d']
#UCI datasets: ['boston','concrete', 'energy', 'powerplant',  'wine', 'yacht', 'protein', 'navalC']
dataset='foong' 

setup_ = get_setup(dataset) #get a module
setup=setup_.Setup(device) #get an object

x_train, y_train=setup.train_data() #scaled_data
x_test, y_test=setup.test_data()
input_dim=x_train.shape[1]
output_dim=y_train.shape[1]
print('input dim: {}'.format(input_dim))
print('output dim: {}'.format(output_dim))
print('nb of train samples: {}'.format(len(x_train)))

#scalar used to scale the train target to have std=1
std_y_train = torch.tensor(1.)
if hasattr(setup, '_scaler_y'):
    print('scaler target: {}'.format(setup._scaler_y.scale_.item()))
    std_y_train=torch.tensor(setup._scaler_y.scale_, device=device).squeeze().float()

input dim: 1
output dim: 1
nb of train samples: 120


# Predictive model #

In [10]:
#predictive model
layerwidth=50
nblayers=1
activation=nn.ReLU()



# Results on test #

In [20]:

#example
nb_predictors=100
nb_test_points=5

y_pred=torch.randn(100,5,1) #nb_predictors x nb_tests x 1

sigma_noise=0.5+torch.rand(100,5,1) #nb_predictors x nb_tests x 1 if heteroscedastic
#sigma_noise=0.5+torch.rand(100,1,1) #nb_predictors x 1 x 1 if homeoscedastic depending on predictors
#sigma_noise=0.5+torch.rand(1,1,1) # 1 x 1 x 1 if homeoscedastic uniform on predictors

y_test=2*torch.randn(5,1)

#calculate metrics
#(RMSE, sqrt(std on test)), 
#(LPP (loglikelihood), std on test), (bigger is better)
# PICP Predictive Interval Covering Probability (theoretical best value 0.95)
# MPIW, Mean Predictive Interval Width (smaller is better)
evaluate_metrics(y_pred,sigma_noise, y_test, std_y_train, device='cpu', std=True)

{'RMSE': (0.7325465679168701, 0.9821411967277527),
 'LPP': (-1.3667142391204834, 0.2508379817008972),
 'PICP': 1.0,
 'MPIW': 3.7475361824035645}